In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from rosbag2_py import SequentialReader, StorageOptions, ConverterOptions
from rclpy.serialization import deserialize_message
from nav_msgs.msg import Odometry
import rclpy


In [ ]:
def read_odom_messages(bag_path):
    storage_options = StorageOptions(uri=bag_path, storage_id='sqlite3')
    converter_options = ConverterOptions(input_serialization_format='cdr', output_serialization_format='cdr')

    reader = SequentialReader()
    reader.open(storage_options, converter_options)
    topics = reader.get_all_topics_and_types()

    odom_msgs = []
    while reader.has_next():
        (topic, data, t) = reader.read_next()
        if topic == '/odom':
            msg = deserialize_message(data, Odometry)
            odom_msgs.append({
                'timestamp': t * 1e-9,
                'x': msg.pose.pose.position.x,
                'y': msg.pose.pose.position.y,
                'z': msg.pose.pose.position.z,
                'linear_x': msg.twist.twist.linear.x,
                'angular_z': msg.twist.twist.angular.z
            })

    return pd.DataFrame(odom_msgs)


In [ ]:
bag_path = "/path/to/your/ros2_bag_folder"  # Folder, not the .db3 file
df_odom = read_odom_messages(bag_path)
df_odom.head()


In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(df_odom['timestamp'], df_odom['x'], label='X')
plt.plot(df_odom['timestamp'], df_odom['y'], label='Y')
plt.xlabel("Time [s]")
plt.ylabel("Position [m]")
plt.legend()
plt.title("Odometry Position Over Time")
plt.grid()
plt.show()


In [ ]:
plt.figure(figsize=(6, 6))
plt.plot(df_odom['x'], df_odom['y'], label="Path")
plt.xlabel("X [m]")
plt.ylabel("Y [m]")
plt.title("2D Robot Path from /odom")
plt.grid()
plt.axis("equal")
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(df_odom['timestamp'], df_odom['linear_x'], label='Linear X Velocity')
plt.plot(df_odom['timestamp'], df_odom['angular_z'], label='Angular Z Velocity')
plt.xlabel("Time [s]")
plt.ylabel("Velocity [m/s] or [rad/s]")
plt.legend()
plt.title("Odometry Velocity Over Time")
plt.grid()
plt.show()


If the Map is Not Growing in RViz:
|__Checkpoint | 	Expected Sign
|/odom topic rate  |	~10 Hz or consistent timestamps
|x, y, theta changes over time |	Values must change if robot is moving
|Linear/Angular velocities non-zero |	Should spike during motion
|TF Tree	| Must have odom → base_link → sensor
|Lidar Scan Topic /scan |	Confirm scan is active + non-zero ranges
|SLAM Toolbox Input |	Uses TF + /scan → if /odom wrong, SLAM gets confused
|No conflicting TFs |	Use ros2 run tf2_tools view_frames to debug
